In [1]:
#default_exp model_hub

# Interacting with HuggingFace and Flair, Model Zoo
> An interactive API for model lookup within HuggingFace and Flair

In [2]:
#hide
from nbverbose.showdoc import *
from fastcore.test import *

In [3]:
#export
from fastcore.basics import Self, merge
from fastcore.utils import dict2obj, obj2dict, mk_class
from fastai.torch_core import apply
from huggingface_hub.hf_api import ModelInfo, HfApi

from typing import List, Dict

## Tasks

`HF_TASKS` and `FLAIR_TASKS` are namespace objects that can enable tab-completion when searching for specific tasks within the `HFModelHub` and `FlairModelHub`

In [4]:
#exporti
_hf_tasks = {
    'FILL_MASK':'fill-mask',
    'QUESTION_ANSWERING':'question-answering',
    'SUMMARIZATION':'summarization',
    'TABLE_QUESTION_ANSWERING':'table-question-answering',
    'TEXT_CLASSIFICATION':'text-classification',
    'TEXT_GENERATION':'text-generation',
    'TEXT2TEXT_GENERATION':'text2text-generation',
    'TOKEN_CLASSIFICATION':'token-classification',
    'TRANSLATION':'translation',
    'ZERO_SHOT_CLASSIFICATION':'zero-shot-classification',
    'CONVERSATIONAL':'conversational',
    'TEXT_TO_SPEECH':'text-to-speech',
    'AUTOMATIC_SPEECH_RECOGNITION':'automatic-speech-recognition',
    'AUDIO_SOURCE_SEPERATION':'audio-source-seperation',
    'VOICE_ACTIVITY_DETECTION':'voice-activity-detection'}

In [5]:
#export
mk_class('HF_TASKS', **_hf_tasks,
        doc="A list of all HuggingFace tasks for valid API lookup as attribtues to get tab-completion and typo-proofing")

In [6]:
#export
_all_ = ['HF_TASKS']

In [7]:
show_doc(HF_TASKS, title_level=4)

<h4 id="HF_TASKS" class="doc_header"><code>class</code> <code>HF_TASKS</code><a href="" class="source_link" style="float:right">[source]</a></h4>

> <code>HF_TASKS</code>(**\*`args`**, **\*\*`kwargs`**)

A list of all HuggingFace tasks for valid API lookup as attribtues to get tab-completion and typo-proofing

**Parameters:**


 - **`args`** : *`<class 'inspect._empty'>`*

 - **`kwargs`** : *`<class 'inspect._empty'>`*


In [8]:
#hide_input
print(f'Possible tasks:')
for val in _hf_tasks.values():
    print(f'* {val}')

Possible tasks:
* fill-mask
* question-answering
* summarization
* table-question-answering
* text-classification
* text-generation
* text2text-generation
* token-classification
* translation
* zero-shot-classification
* conversational
* text-to-speech
* automatic-speech-recognition
* audio-source-seperation
* voice-activity-detection


In [9]:
#exporti
_flair_tasks = {
    'NAMED_ENTITY_RECOGNITION':'ner',
    'PHRASE_CHUNKING':'chunk',
    'VERB_DISAMBIGUATION':'frame',
    'PART_OF_SPEECH_TAGGING':'pos',
    'UNIVERSAL_PART_OF_SPEECH_TAGGING':'upos',
    'EMBEDDINGS':'embeddings',
    
}

In [10]:
#export
mk_class('FLAIR_TASKS', **_flair_tasks,
        doc="A list of all Flair tasks for valid API lookup as attributes to get tab-completion and typo-proofing")

In [11]:
#export
_all_ = ['FLAIR_TASKS']

In [12]:
show_doc(FLAIR_TASKS, title_level=4)

<h4 id="FLAIR_TASKS" class="doc_header"><code>class</code> <code>FLAIR_TASKS</code><a href="" class="source_link" style="float:right">[source]</a></h4>

> <code>FLAIR_TASKS</code>(**\*`args`**, **\*\*`kwargs`**)

A list of all Flair tasks for valid API lookup as attributes to get tab-completion and typo-proofing

**Parameters:**


 - **`args`** : *`<class 'inspect._empty'>`*

 - **`kwargs`** : *`<class 'inspect._empty'>`*


In [13]:
#hide_input
print(f'Possible tasks:')
for val in _flair_tasks.values():
    print(f'* {val}')

Possible tasks:
* ner
* chunk
* frame
* pos
* upos
* embeddings


In [17]:
#export
class HFModelResult:
    """
    A very basic class for storing a HuggingFace model returned through an API request
    
    They have 4 properties:
      - `name`: The `modelId` from the `modelInfo`. This also includes the model author's name, such as "IlyaGusev/mbart_ru_sum_gazeta"
      - `tags`: Any tags that were included in `HuggingFace` in relation to the model. 
      - `tasks`: These are the tasks dictated for the model.
    """
    def __init__(
        self, 
        model_info: ModelInfo # `ModelInfo` object from HuggingFace model hub
    ):
        self.info = model_info
        
    def __repr__(self): return f'Model Name: {self.name}, Tasks: [' + ', '.join(self.tasks) + ']'
    
    @property
    def name(self): return self.info.modelId

    @property
    def tags(self): return self.info.tags
    
    @property
    def tasks(self): 
        if self.info.pipeline_tag:
            all_tasks = [self.info.pipeline_tag]
            for tag in self.tags:
                if (tag in _hf_tasks.values()) and (tag not in all_tasks):
                    all_tasks += [tag]
        else: all_tasks = []
        all_tasks.sort()
        return all_tasks
    
    def to_dict(
        self
    ) -> dict: # Dictionary with keys `model_name`, `tags`, `tasks`, `model_info` 
        "Returns `HFModelResult` as a dictionary"
        return {'model_name':self.name, 'tags':self.tags, 'tasks':self.tasks, 'model_info':self.info}

In [18]:
show_doc(HFModelResult)

<h2 id="HFModelResult" class="doc_header"><code>class</code> <code>HFModelResult</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>HFModelResult</code>(**`model_info`**:`ModelInfo`)

A very basic class for storing a HuggingFace model returned through an API request

They have 4 properties:
  - `name`: The `modelId` from the `modelInfo`. This also includes the model author's name, such as "IlyaGusev/mbart_ru_sum_gazeta"
  - `tags`: Any tags that were included in `HuggingFace` in relation to the model. 
  - `tasks`: These are the tasks dictated for the model.

**Parameters:**


 - **`model_info`** : *`<class 'huggingface_hub.hf_api.ModelInfo'>`*	<p>`ModelInfo` object from HuggingFace model hub</p>



We look inside of `modelInfo.pipeline_tag` as well as the `tags` for if there is any overlap

In [19]:
show_doc(HFModelResult.to_dict)

<h4 id="HFModelResult.to_dict" class="doc_header"><code>HFModelResult.to_dict</code><a href="__main__.py#L36" class="source_link" style="float:right">[source]</a></h4>

> <code>HFModelResult.to_dict</code>()

Returns [`HFModelResult`](/adaptnlp/model_hub.html#HFModelResult) as a dictionary



**Returns**:
	
 * *`<class 'dict'>`*	<p>Dictionary with keys `model_name`, `tags`, `tasks`, `model_info`</p>



In [20]:
#export
class HFModelHub:
    "A class for interacting with the HF model hub API, and searching for models by name or task"
    
    def __init__(
        self, 
        username:str=None, # Your HuggingFace username
        password:str=None # Your HuggingFace password
    ):
        self.api = HfApi()
        if username and password:
            self.token = self.api.login(username, password)
        elif username or password:
            print('Only a username or password was entered. You should include both to get authorized access')
        
    def _format_results(
        self, 
        results:list, # A list of HuggingFace API results
        as_dict:bool=False, # Whether to return as a dictionary or list
        user_uploaded:bool=False # Whether to filter out user-uploaded results
    ) -> (List[HFModelResult], Dict[str, HFModelResult]): # A list of `HFModelResult`s
        "Takes raw HuggingFace API results and makes them easier to read and work with"
        results = apply(HFModelResult, results)
        if not user_uploaded:
            results = [r for r in results if '/' not in r.name]
        if as_dict:
            dicts = apply(Self.to_dict(), results)
            results = {m['model_name'] : m for m in dicts}
        return results
        
    def search_model_by_task(
        self, 
        task:str, # A valid task to search in the HuggingFace hub for
        as_dict:bool=False, # Whether to return as a dictionary or list
        user_uploaded:bool=False # Whether to filter out user-uploaded results
    ) -> (List[HFModelResult], Dict[str, HFModelResult]): # A list of `HFModelResult`s
        "Searches HuggingFace Model API for all pretrained models relating to `task`"
        if task not in _hf_tasks.values():
            raise ValueError(f'''`{task}` is not a valid task. 
            
            Please choose a valid one available from HuggingFace: (https://huggingface.co/transformers/task_summary.html) 
            Or with the `HF_TASKS` object''')
        models = self.api.list_models(task)
        return self._format_results(models, as_dict, user_uploaded)
    
    def search_model_by_name(
        self, 
        name:str, # A valid model name 
        as_dict:bool=False, # Whether to return as a dictionary or list
        user_uploaded:bool=False # Whether to filter out user-uploaded results
    ) -> (List[HFModelResult], Dict[str, HFModelResult]): # A list of `HFModelResult`s
        "Searches HuggingFace Model API for all pretrained models containing `name`"
        if user_uploaded:
            models = self.api.list_models()
            models = self._format_results(models, as_dict, user_uploaded)
            models = [m for m in models if name in m.name]
            
        else:
            models = self.api.list_models(name)
            models = self._format_results(models, as_dict, user_uploaded)
        return models

The model search hub creates a friendly end-user API when searching through HuggingFace (and Flair, as we will see later). Usage is extremely simple as well.

In [21]:
show_doc(HFModelHub.search_model_by_task)

<h4 id="HFModelHub.search_model_by_task" class="doc_header"><code>HFModelHub.search_model_by_task</code><a href="__main__.py#L31" class="source_link" style="float:right">[source]</a></h4>

> <code>HFModelHub.search_model_by_task</code>(**`task`**:`str`, **`as_dict`**:`bool`=*`False`*, **`user_uploaded`**:`bool`=*`False`*)

Searches HuggingFace Model API for all pretrained models relating to `task`

**Parameters:**


 - **`task`** : *`<class 'str'>`*	<p>A valid task to search in the HuggingFace hub for</p>


 - **`as_dict`** : *`<class 'bool'>`*, *optional*	<p>Whether to return as a dictionary or list</p>


 - **`user_uploaded`** : *`<class 'bool'>`*, *optional*	<p>Whether to filter out user-uploaded results</p>



**Returns**:
	
 * *`(typing.List[__main__.HFModelResult], typing.Dict[str, __main__.HFModelResult])`*	<p>A list of `HFModelResult`s</p>



This will return a list of models available for a particular class. A few usage examples are below:

In [22]:
hub = HFModelHub()
models = hub.search_model_by_task('summarization', user_uploaded=False, as_dict=False)
models

[Model Name: t5-11b, Tasks: [summarization, text2text-generation, translation],
 Model Name: t5-3b, Tasks: [summarization, text2text-generation, translation],
 Model Name: t5-base, Tasks: [summarization, text2text-generation, translation],
 Model Name: t5-large, Tasks: [summarization, text2text-generation, translation],
 Model Name: t5-small, Tasks: [summarization, text2text-generation, translation]]

In [23]:
#hide
test_eq(models[0].name, 't5-11b')
test_eq(models[0].tasks, ['summarization', 'text2text-generation', 'translation'])

We can also search for any user-uploaded models from the community too:

In [24]:
models = hub.search_model_by_task('summarization', user_uploaded=True)
models[:10]

[Model Name: t5-11b, Tasks: [summarization, text2text-generation, translation],
 Model Name: t5-3b, Tasks: [summarization, text2text-generation, translation],
 Model Name: t5-base, Tasks: [summarization, text2text-generation, translation],
 Model Name: t5-large, Tasks: [summarization, text2text-generation, translation],
 Model Name: t5-small, Tasks: [summarization, text2text-generation, translation],
 Model Name: Callidior/bert2bert-base-arxiv-titlegen, Tasks: [summarization, text2text-generation],
 Model Name: IlyaGusev/mbart_ru_sum_gazeta, Tasks: [summarization, text2text-generation],
 Model Name: IlyaGusev/rubert_telegram_headlines, Tasks: [summarization, text2text-generation],
 Model Name: LeoCordoba/beto2beto-ccnews-titles-es, Tasks: [summarization, text2text-generation],
 Model Name: LeoCordoba/beto2beto-mlsum, Tasks: [summarization, text2text-generation]]

In [25]:
#hide
model = models[5]
author, name = model.name.split('/')
test_eq(author, 'Callidior')
test_eq(name, 'bert2bert-base-arxiv-titlegen')
test_eq(model.tasks, ['summarization', 'text2text-generation'])

There are also cases where a `dict` may be easier to work with (perhaps utilizing a network API, or ease of use for some). We can instead return a dictionary of `HFModelResult` objects too by passing `as_dict=True` to any search call:

In [26]:
models = hub.search_model_by_task('summarization', as_dict=True);
models['t5-11b']

{'model_name': 't5-11b',
 'tags': ['pytorch',
  'tf',
  't5',
  'lm-head',
  'seq2seq',
  'en',
  'fr',
  'ro',
  'de',
  'dataset:c4',
  'arxiv:1910.10683',
  'transformers',
  'summarization',
  'translation',
  'license:apache-2.0',
  'text2text-generation'],
 'tasks': ['summarization', 'text2text-generation', 'translation'],
 'model_info': ModelInfo: {
 	modelId: t5-11b
 	sha: b8fabb39157e07006719ced3f9b9b91a4344317d
 	lastModified: 2021-03-18T01:58:45.000Z
 	tags: ['pytorch', 'tf', 't5', 'lm-head', 'seq2seq', 'en', 'fr', 'ro', 'de', 'dataset:c4', 'arxiv:1910.10683', 'transformers', 'summarization', 'translation', 'license:apache-2.0', 'text2text-generation']
 	pipeline_tag: translation
 	siblings: [ModelFile(rfilename='.gitattributes'), ModelFile(rfilename='README.md'), ModelFile(rfilename='config.json'), ModelFile(rfilename='pytorch_model.bin'), ModelFile(rfilename='spiece.model'), ModelFile(rfilename='tf_model.h5'), ModelFile(rfilename='tokenizer.json')]
 	config: None
 	private

In [27]:
#hide
test_eq(models.keys(), ['t5-11b', 't5-3b', 't5-base', 't5-large', 't5-small'])
test_eq(models['t5-11b'].keys(), ['model_name', 'tags', 'tasks', 'model_info'])

This will return a dictionary of the name, the HuggingFace tags affiliated with the model, the dictated tasks, and an instance of `huggingface_hub`'s `ModelInfo`.

In [28]:
show_doc(HFModelHub.search_model_by_name)

<h4 id="HFModelHub.search_model_by_name" class="doc_header"><code>HFModelHub.search_model_by_name</code><a href="__main__.py#L46" class="source_link" style="float:right">[source]</a></h4>

> <code>HFModelHub.search_model_by_name</code>(**`name`**:`str`, **`as_dict`**:`bool`=*`False`*, **`user_uploaded`**:`bool`=*`False`*)

Searches HuggingFace Model API for all pretrained models containing `name`

**Parameters:**


 - **`name`** : *`<class 'str'>`*	<p>A valid model name</p>


 - **`as_dict`** : *`<class 'bool'>`*, *optional*	<p>Whether to return as a dictionary or list</p>


 - **`user_uploaded`** : *`<class 'bool'>`*, *optional*	<p>Whether to filter out user-uploaded results</p>



**Returns**:
	
 * *`(typing.List[__main__.HFModelResult], typing.Dict[str, __main__.HFModelResult])`*	<p>A list of `HFModelResult`s</p>



With `search_model_by_name` you're allowed a bit more freedom in what you wish to search for. `search_model_by_name` downloads the entire list of models from `HuggingFace` then performs partial string matching. As a result you can search for all models by a particular user by doing:

In [29]:
hub.search_model_by_name('Callidior', user_uploaded=True)

[Model Name: Callidior/bert2bert-base-arxiv-titlegen, Tasks: [summarization]]

Or (as implied by the function name) any model type itself:

In [30]:
hub.search_model_by_name('gpt2', user_uploaded=True)[5:10]

[Model Name: 850886470/xxy_gpt2_chinese, Tasks: [],
 Model Name: ComCom-Dev/gpt2-bible-test, Tasks: [],
 Model Name: DHBaek/gpt2-stackoverflow-question-contents-generator, Tasks: [text-generation],
 Model Name: DeepESP/gpt2-spanish, Tasks: [text-generation],
 Model Name: Fabby/gpt2-english-light-novel-titles, Tasks: []]

In [31]:
#exporti
# Flair models originating from:
# https://github.com/flairNLP/flair/blob/master/flair/models/text_classification_model.py#L483
# and: https://github.com/flairNLP/flair/blob/master/flair/models/sequence_tagger_model.py#L1053
# and: https://github.com/flairNLP/flair/blob/master/flair/embeddings/token.py#L406
_flair_models = {
    'de-offensive-language' : ['text-classification'],
    'sentiment' : ['text-classification'],
    'en-sentiment' : ['text-classification'],
    'sentiment-fast' : ['text-classification'],
    'communicative-functions' : ['text-classification'],
    'tars-base' : ['text-classification'],
    # English Named Entity Recognition Models (NER)
    'ner' : ['token-classification'],
    'ner-pooled' : ['token-classification'],
    'ner-fast' : ['token-classification'],
    'ner-ontonotes' : ['token-classification'],
    'ner-ontonotes-fast' : ['token-classification'],
    # Multilingual NER models
    'ner-multi' : ['token-classification'],
    'multi-ner' : ['token-classification'],
    'ner-multi-fast' : ['token-classification'],
    # English POS models
    'upos' : ['token-classification'],
    'upos-fast' : ['token-classification'],
    'pos' : ['token-classification'],
    'pos-fast' : ['token-classification'],
    # Multilingual POS models
    'pos-multi' : ['token-classification'],
    'multi-pos' : ['token-classification'],
    'pos-multi-fast' : ['token-classification'],
    'multi-pos-fast' : ['token-classification'],
    # English SRL models
    'frame' : ['token-classification'],
    'frame-fast' : ['token-classification'],
    # English chunking models
    'chunk' : ['token-classification'],
    'chunk-fast' : ['token-classification'],
    # Danish models
    'da-pos' : ['token-classification'],
    'da-ner' : ['token-classification'],
    # German models
    'de-pos' : ['token-classification'],
    'de-pos-tweets' : ['token-classification'],
    'de-ner' : ['token-classification'],
    'de-ner-germeval' : ['token-classification'],
    'de-ler' : ['token-classification'],
    'de-ner-legal' : ['token-classification'],
    # French models
    'fr-ner' : ['token-classification'],
    # Dutch models
    'nl-ner' : ['token-classification'],
    'nl-ner-rnn' : ['token-classification'],
    # Malayalam models
    'ml-pos' : ['token-classification'],
    'ml-upos' : ['token-classification'],
    # Portuguese models
    'pt-pos-clinical' : ['token-classification'],
    # Keyphrase models
    'keyphrase' : ['token-classification'],
    'negation-speculation' : ['token-classification'],
    # Biomedical
    'hunflair-paper-cellline' : ['token-classification'],
    'hunflair-paper-chemical' : ['token-classification'],
    'hunflair-paper-disease' : ['token-classification'],
    'hunflair-paper-gene' : ['token-classification'],
    'hunflair-paper-species' : ['token-classification'],
    'hunflair-cellline' : ['token-classification'],
    'hunflair-chemical' : ['token-classification'],
    'hunflair-disease' : ['token-classification'],
    'hunflair-gene' : ['token-classification'],
    'hunflair-species' : ['token-classification'],
    # Embeddings
    # multilingual models
    "multi-forward":['embeddings'],
    "multi-backward":['embeddings'],
    "multi-v0-forward":['embeddings'],
    "multi-v0-backward":['embeddings'],
    "multi-forward-fast":['embeddings'],
    "multi-backward-fast":['embeddings'],
    # English models
    "en-forward":['embeddings'],
    "en-backward":['embeddings'],
    "en-forward-fast":['embeddings'],
    "en-backward-fast":['embeddings'],
    "news-forward":['embeddings'],
    "news-backward":['embeddings'],
    "news-forward-fast":['embeddings'],
    "news-backward-fast":['embeddings'],
    "mix-forward":['embeddings'],
    "mix-backward":['embeddings'],
    # Arabic
    "ar-forward":['embeddings'],
    "ar-backward":['embeddings'],
    # Bulgarian
    "bg-forward-fast":['embeddings'],
    "bg-backward-fast":['embeddings'],
    "bg-forward":['embeddings'],
    "bg-backward":['embeddings'],
    # Czech
    "cs-forward":['embeddings'],
    "cs-backward":['embeddings'],
    "cs-v0-forward":['embeddings'],
    "cs-v0-backward":['embeddings'],
    # Danish
    "da-forward":['embeddings'],
    "da-backward":['embeddings'],
    # German
    "de-forward":['embeddings'],
    "de-backward":['embeddings'],
    "de-historic-ha-forward":['embeddings'],
    "de-historic-ha-backward":['embeddings'],
    "de-historic-wz-forward":['embeddings'],
    "de-historic-wz-backward":['embeddings'],
    "de-historic-rw-forward":['embeddings'],
    "de-historic-rw-backward":['embeddings'],
    # Spanish
    "es-forward":['embeddings'],
    "es-backward":['embeddings'],
    "es-forward-fast":['embeddings'],
    "es-backward-fast":['embeddings'],
    # Basque
    "eu-forward":['embeddings'],
    "eu-backward":['embeddings'],
    "eu-v1-forward":['embeddings'],
    "eu-v1-backward":['embeddings'],
    "eu-v0-forward":['embeddings'],
    "eu-v0-backward":['embeddings'],
    # Persian
    "fa-forward":['embeddings'],
    "fa-backward":['embeddings'],
    # Finnish
    "fi-forward":['embeddings'],
    "fi-backward":['embeddings'],
    # French
    "fr-forward":['embeddings'],
    "fr-backward":['embeddings'],
    # Hebrew
    "he-forward":['embeddings'],
    "he-backward":['embeddings'],
    # Hindi
    "hi-forward":['embeddings'],
    "hi-backward":['embeddings'],
    # Croatian
    "hr-forward":['embeddings'],
    "hr-backward":['embeddings'],
    # Indonesian
    "id-forward":['embeddings'],
    "id-backward":['embeddings'],
    # Italian
    "it-forward":['embeddings'],
    "it-backward":['embeddings'],
    # Japanese
    "ja-forward":['embeddings'],
    "ja-backward":['embeddings'],
    # Malayalam
    "ml-forward":['embeddings'],
    "ml-backward":['embeddings'],
    # Dutch
    "nl-forward":['embeddings'],
    "nl-backward":['embeddings'],
    "nl-v0-forward":['embeddings'],
    "nl-v0-backward":['embeddings'],
    # Norwegian
    "no-forward":['embeddings'],
    "no-backward":['embeddings'],
    # Polish
    "pl-forward":['embeddings'],
    "pl-backward":['embeddings'],
    "pl-opus-forward":['embeddings'],
    "pl-opus-backward":['embeddings'],
    # Portuguese
    "pt-forward":['embeddings'],
    "pt-backward":['embeddings'],
    # Pubmed
    "pubmed-forward":['embeddings'],
    "pubmed-backward":['embeddings'],
    "pubmed-2015-forward":['embeddings'],
    "pubmed-2015-backward":['embeddings'],
    # Slovenian
    "sl-forward":['embeddings'],
    "sl-backward":['embeddings'],
    "sl-v0-forward":['embeddings'],
    "sl-v0-backward":['embeddings'],
    # Swedish
    "sv-forward":['embeddings'],
    "sv-backward":['embeddings'],
    "sv-v0-forward":['embeddings'],
    "sv-v0-backward":['embeddings'],
    # Tamil
    "ta-forward":['embeddings'],
    "ta-backward":['embeddings'],
    # CLEF HIPE Shared task
    "de-impresso-hipe-v1-forward":['embeddings'],
    "de-impresso-hipe-v1-backward":['embeddings'],
    "en-impresso-hipe-v1-forward":['embeddings'],
    "en-impresso-hipe-v1-backward":['embeddings'],
    "fr-impresso-hipe-v1-forward":['embeddings'],
    "fr-impresso-hipe-v1-backward":['embeddings']    
}

In [32]:
#export
FLAIR_MODELS = [ModelInfo(f'flairNLP/{key}', pipeline_tag=val[0]) for key,val in _flair_models.items()]

Flair has a series of extra models available for use that are not available through HuggingFace such as `sentiment`, `communicative-functions`, and more. `FLAIR_MODELS` is a convience holder for quick lookup of these models (as no such list is easily available currently). When shown as results on the API they will be given the same `flair` prefix for convience.

In [33]:
#export
class FlairModelResult(HFModelResult):
    """
    A version of `HFModelResult` for Flair specifically. 
    
    Includes which backend the model was found (such as on HuggingFace or Flair's private model list)
    """
    
    def __init__(
        self, 
        model_info: ModelInfo # ModelInfo object from HuggingFace model hub
    ):
        if 'flairNLP' in model_info.modelId:
            self.from_hf = False
        else:
            self.from_hf = True
        super().__init__(model_info)
        
    def __repr__(self): return f'Model Name: {self.name.replace("flairNLP", "flair")}, Tasks: [' + ', '.join(self.tasks) + ']' + f', Source: {self.source}'
    
    @property
    def source(self):
        if self.from_hf: return "HuggingFace Model Hub"
        else: return "Flair's Private Model Hub"

In [34]:
#export
class FlairModelHub:
    "A class for interacting with the HF model hub API, and searching for Flair models by name or task"
    
    def __init__(
        self, 
        username:str=None, # HuggingFace username
        password:str=None # HuggingFace password
    ):
        self.api = HfApi()
        if username and password:
            self.token = self.api.login(username, password)
        elif username or password:
            print('Only a username or password was entered. You should include both to get authorized access')
        self.models = self.api.list_models('flair') + FLAIR_MODELS
        
    def _format_results(
        self, 
        results:list, # A list of HuggingFace API results
        as_dict:bool=False, # Whether to return as a dictionary or list
        user_uploaded:bool=False # Whether to filter out user-uploaded results
    ) -> (List[FlairModelResult], Dict[str, FlairModelResult]): # A list of `FlairModelResult`s
        "Takes raw HuggingFace API results and makes them easier to read and work with"
        results = apply(FlairModelResult, results)
        if not user_uploaded:
            results = [r for r in results if 'flair/' in r.name or 'flairNLP/' in r.name]
        if as_dict:
            dicts = apply(Self.to_dict(), results)
            results = {m['model_name'] : m for m in dicts}
        return results
    
    def search_model_by_name(
        self, 
        name:str, # A valid model name 
        as_dict:bool=False, # Whether to return as a dictionary or list
        user_uploaded:bool=False # Whether to filter out user-uploaded results
    ) -> (List[FlairModelResult], Dict[str, FlairModelResult]): # A list of `FlairModelResult`s
        "Searches HuggingFace Model API for all flair models containing `name`"
        models = [m for m in self.models if name in m.modelId]
        return self._format_results(models, as_dict, user_uploaded)
    
    def search_model_by_task(
        self, 
        task:str, 
        as_dict=False, 
        user_uploaded=False
    ) -> (List[FlairModelResult], Dict[str, FlairModelResult]): # A list of `FlairModelResult`s
        "Searches HuggingFace Model API for all flair models for `task`"
        if (task not in _flair_tasks.values()) and (task != ''):
            raise ValueError(f'''`{task}` is not a valid task. 
            
            Please choose a valid one available from Flair: (https://huggingface.co/flair) 
            Or with the `FLAIR_TASKS` object''')
        models = [m for m in self.models if task in m.modelId or task == m.pipeline_tag]
        return self._format_results(models, as_dict, user_uploaded)

In [35]:
show_doc(FlairModelHub)

<h2 id="FlairModelHub" class="doc_header"><code>class</code> <code>FlairModelHub</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>FlairModelHub</code>(**`username`**:`str`=*`None`*, **`password`**:`str`=*`None`*)

A class for interacting with the HF model hub API, and searching for Flair models by name or task

**Parameters:**


 - **`username`** : *`<class 'str'>`*, *optional*	<p>HuggingFace username</p>


 - **`password`** : *`<class 'str'>`*, *optional*	<p>HuggingFace password</p>



`FlairModelHub` is extremely similar to `HFModelHub`, with the two differences being that it will **only** return `Flair` models, and it has access to the *other* Flair models available that can't be accessed through the HuggingFace model hub

In [36]:
hub = FlairModelHub()

In [37]:
show_doc(FlairModelHub.search_model_by_name)

<h4 id="FlairModelHub.search_model_by_name" class="doc_header"><code>FlairModelHub.search_model_by_name</code><a href="__main__.py#L32" class="source_link" style="float:right">[source]</a></h4>

> <code>FlairModelHub.search_model_by_name</code>(**`name`**:`str`, **`as_dict`**:`bool`=*`False`*, **`user_uploaded`**:`bool`=*`False`*)

Searches HuggingFace Model API for all flair models containing `name`

**Parameters:**


 - **`name`** : *`<class 'str'>`*	<p>A valid model name</p>


 - **`as_dict`** : *`<class 'bool'>`*, *optional*	<p>Whether to return as a dictionary or list</p>


 - **`user_uploaded`** : *`<class 'bool'>`*, *optional*	<p>Whether to filter out user-uploaded results</p>



**Returns**:
	
 * *`(typing.List[__main__.FlairModelResult], typing.Dict[str, __main__.FlairModelResult])`*	<p>A list of `FlairModelResult`s</p>



`seach_model_by_name` will also let you search for models without needing the `flair` prefix, such as:

In [38]:
hub.search_model_by_name('sentiment')

[Model Name: flair/sentiment, Tasks: [text-classification], Source: Flair's Private Model Hub,
 Model Name: flair/en-sentiment, Tasks: [text-classification], Source: Flair's Private Model Hub,
 Model Name: flair/sentiment-fast, Tasks: [text-classification], Source: Flair's Private Model Hub]

In [39]:
#hide
test_eq(0, len(hub.search_model_by_name('gpt')))

In [40]:
show_doc(FlairModelHub.search_model_by_task)

<h4 id="FlairModelHub.search_model_by_task" class="doc_header"><code>FlairModelHub.search_model_by_task</code><a href="__main__.py#L42" class="source_link" style="float:right">[source]</a></h4>

> <code>FlairModelHub.search_model_by_task</code>(**`task`**:`str`, **`as_dict`**=*`False`*, **`user_uploaded`**=*`False`*)

Searches HuggingFace Model API for all flair models for `task`

**Parameters:**


 - **`task`** : *`<class 'str'>`*

 - **`as_dict`** : *`<class 'bool'>`*, *optional*

 - **`user_uploaded`** : *`<class 'bool'>`*, *optional*


**Returns**:
	
 * *`(typing.List[__main__.FlairModelResult], typing.Dict[str, __main__.FlairModelResult])`*	<p>A list of `FlairModelResult`s</p>



Since we have a `FLAIR_TASKS` object declared earlier, we can utilize it when searching for models by a task. Similar to `search_model_by_name` you should not include `flair/` in your search results, and instead search through the task key such as `ner` or `FLAIR_TASKS.NAMED_ENTITY_RECOGNITION`

In [41]:
#hide
models = hub.search_model_by_task('ner')
models = [m for m in models if m.source == "Flair's Private Model Hub"]
test_eq(len(models), 15)